In [1]:
import nltk
import torch
import pandas as pd
from evaluate import load
from bert_score import score
from nltk.corpus import stopwords
from transformers import AutoModel, AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity

d:\Developer\LangChain\chatbot-edital\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('predictions.csv')
df

,pergunta,texto_original,texto_gerado
0,1) Quais são as atividades especializadas atri...,"Coletar, inserir, transformar, qualificar e en...","Segundo o contexto fornecido, as atividades pr..."
1,2) Quais são as atividades previstas para um A...,"Gerenciar Redes, Serviços de Rede e Suporte de...","De acordo com o contexto fornecido, as ativida..."
2,3) Quais são as atividades previstas para um E...,Informação não consta no presente edital.,Não há informações sobre o cargo de Especialis...
3,4) Para o cargo “Analista de Ciência e Tecnolo...,"5 (Ampla Concorrência), 1 (PcD) e 1 (Negro), t...",Não há informações sobre o quantitativo de vag...
4,5) Qual é o percentual de vagas reservadas par...,Às pessoas com deficiência são reservadas 5% (...,"Segundo o texto, as pessoas com deficiência (P..."
5,6) Qual é a fórmula usada para calcular a médi...,Sob a seguinte fórmula: Mp= (p1.x1+p2.x2+...+p...,A fórmula usada para calcular a média ponderad...
6,7) Qual o conteúdo programático para os candid...,Informação não consta no presente edital.,resposta não está disponível no contexto forne...
7,8) Qual o valor do salário bruto inicial de um...,"R$4.875,18 (Vencimento Básico) + R$5.606,46 (R...","Infelizmente, não há informação específica sob..."
8,9) Qual é o limite máximo de pontos que os can...,A pontuação máxima atribuída para Prova de Tít...,"Segundo o contexto, o limite máximo de pontos ..."
9,10) Para o cargo “Agente de Combate de Endemia...,Epidemiologia. Endemias e epidemias. Etiologia...,"Segundo o contexto, os conhecimentos específic..."


In [3]:
previsões = df['texto_gerado'].tolist()
referências = df['texto_original'].tolist()

In [4]:
# Calcular BERTScore para todas as previsões e referências
P, R, F1 = score(previsões, referências, lang='pt')

# Calcular média do BERTScore
media_bert_score = sum(F1) / len(F1)

# Imprimir o resultado
print(f"BERTScore: {media_bert_score}")

BERTScore: 0.7244503498077393


In [5]:
def clean_text(text):
    tokens = nltk.word_tokenize(text.strip(), language="portuguese")
    filtered_tokens = list(filter(lambda w: w not in sw, tokens))
    return " ".join(filtered_tokens).lower()

# Baixar stopwords e pacote punkt do nltk
nltk.download('stopwords')
nltk.download('punkt')

# Configurar stopwords
sw = stopwords.words('portuguese')
sw.extend(['.', ":", ";", "!", "?", "\\", "/", "*", "-", ",", "#", "@"])

# Carregar o CSV
csv_path = 'predictions.csv'   # Substitua pelo caminho do seu CSV
df = pd.read_csv(csv_path)

# Limpar as colunas texto_original e texto_gerado
df['texto_original'] = df['texto_original'].apply(clean_text)
df['texto_gerado'] = df['texto_gerado'].apply(clean_text)

# Preparar dados para avaliação
previsões = list(df['texto_gerado'])
referências = list(df['texto_original'])  # Apenas uma lista de strings, não uma lista de listas

metric_rouge = load('rouge')
result_rouge = metric_rouge.compute(predictions=previsões, references=[[ref] for ref in referências])

meteor = load('meteor')
result_meteor = meteor.compute(predictions=previsões, references=[[ref] for ref in referências])

# Carregar modelo BERT e tokenizer
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=True, padding='max_length', max_len=500)

mean_cosine_similarity = 0
for i in range(df.shape[0]):
    row = df.iloc[i]
    sentence1 = str(row['texto_original'])
    sentence2 = str(row['texto_gerado'])

    # Tokenizar e codificar as sentenças
    inputs1 = tokenizer(sentence1, return_tensors='pt', truncation=True, padding=True)
    inputs2 = tokenizer(sentence2, return_tensors='pt', truncation=True, padding=True)

    # Codificar as sentenças
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    # Extrair embeddings
    embeddings1 = outputs1.last_hidden_state[:, 0, :]
    embeddings2 = outputs2.last_hidden_state[:, 0, :]

    # Calcular a similaridade de cosseno
    similarity_score = cosine_similarity(embeddings1, embeddings2).item()
    mean_cosine_similarity += similarity_score

mean_cosine_similarity /= df.shape[0]

# Criação do DataFrame de métricas
metrics = {
    "metric": ["ROUGE1", "ROUGE2", "ROUGEL", "METEOR", "COSINE SIMILARITY", "BERTScore"],
    "value": [
        result_rouge['rouge1'], result_rouge['rouge2'], result_rouge['rougeL'], result_meteor['meteor'],
        mean_cosine_similarity, media_bert_score
    ]
}

df_metrics = pd.DataFrame(metrics)
print(df_metrics)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


              metric           value
0             ROUGE1        0.288695
1             ROUGE2        0.186691
2             ROUGEL        0.252039
3             METEOR        0.237754
4  COSINE SIMILARITY        0.772391
5          BERTScore  tensor(0.7245)


In [6]:
df_metrics = pd.DataFrame(metrics)
df_metrics

,metric,value
0,ROUGE1,0.288695
1,ROUGE2,0.186691
2,ROUGEL,0.252039
3,METEOR,0.237754
4,COSINE SIMILARITY,0.772391
5,BERTScore,tensor(0.7245)
